In [1]:
#!pip install nsetools
# %pip install beautifulsoup4


In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from pprint import pprint

In [3]:
class Stock:
    def __init__(self , ticker,sector):
        self.ticker = ticker
        self.sector = sector
        self.price = 0.0
        # url to scrape the data from yahoo finance
        self.url = f"https://finance.yahoo.com/quote/{self.ticker}/key-statistics?p={self.ticker}"
        self.data = {}
        
        # all the metrics
        self.metric_aliases = {
            'Market Cap (intraday)': 'market_cap',
            'Beta (5Y Monthly)': 'beta',
            '52 Week High 3': '52_week_high',
            '52 Week Low 3': '52_week_low',
            '50-Day Moving Average 3': '50_day_ma',
            '200-Day Moving Average 3': '200_day_ma',
            'Avg Vol (3 month) 3': 'avg_vol_3m',
            'Avg Vol (10 day) 3': 'avg_vol_10d',
            'Shares Outstanding 5': 'shares_outstanding',
            'Float 8': 'float',
            '% Held by Insiders 1': 'held_by_insiders',
            '% Held by Institutions 1': 'held_by_institutions',
            'Short Ratio (Jan 30, 2023) 4': 'short_ratio',
            'Payout Ratio 4': 'payout_ratio',
            'Profit Margin': 'profit_margin',
            'Operating Margin (ttm)': 'operating_margin',
            'Return on Assets (ttm)': 'return_on_assets',
            'Return on Equity (ttm)': 'return_on_equity',
            'Revenue (ttm)': 'revenue',
            'Revenue Per Share (ttm)': 'revenue_per_share',
            'Gross Profit (ttm)': 'gross_profit',
            'EBITDA ': 'ebitda',
            'Net Income Avi to Common (ttm)': 'net_income',
            'Diluted EPS (ttm)': 'eps',
            'Total Cash (mrq)': 'total_cash',
            'Total Cash Per Share (mrq)': 'cash_per_share',
            'Total Debt (mrq)': 'total_debt',
            'Total Debt/Equity (mrq)': 'debt_to_equity',
            'Current Ratio (mrq)': 'current_ratio',
            'Book Value Per Share (mrq)': 'book_value_per_share',
            'Operating Cash Flow (ttm)': 'operating_cash_flow',
            'Levered Free Cash Flow (ttm)': 'levered_free_cash_flow'
        }
    
    
    
    
    # function to scrape the data
    def scrape_data(self):
        page = requests.get(self.url ,  headers  = self.get_headers())
        soup =  BeautifulSoup(page.content, 'html.parser')
        
        data = {}
        sections = soup.find_all("section" , {'data-test' : 'qsp-statistics'})
        for section in sections:
            rows = section.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    metric =  cols[0].text.strip()
                    if metric in self.metric_aliases:
                        data[self.metric_aliases[metric]] = cols[1].text.strip()
        
        self.data = data
    
    
    # get stock price
    
    def get_stock_price(self):
        try :
            url = f"https://finance.yahoo.com/quote/{self.ticker}"
            response = requests.get(url, headers = self.get_headers())
            soup = BeautifulSoup(response.content , 'html.parser')
            data = soup.find('fin-streamer', {'data-symbol' : self.ticker})
            price = float(data['value'])
            self.price = price
            print(f'Gathered price of {self.ticker}!!!!')
        except:
            print(f'Price not available for {self.ticker}')
            self.price = 0.0
    
    def get_headers(self):
        return {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36"}
    

In [4]:
# symbol =  Stock('AAPL')
# symbol.get_stock_price()
# symbol.scrape_data()
# pprint(symbol.data)
# symbol.price


In [5]:
# Stock Screener Class
class StockScreener:
    def __init__(self,stocks,filters):
        self.stocks  = stocks
        self.filters = filters
    
    # to add data to the respective class
    def add_data(self):
        for stock  in self.stocks:
            stock.scrape_data()
            stock.get_stock_price()
            
    # filtering out the stocks based on the filters
    def apply_filters(self):
        filtered_stock = []
        for stock in self.stocks:
            passed_all_filters = True
            for filter_func in self.filters:
                if not filter_func(stock):
                    passed_all_filters = False
                    break
            if passed_all_filters:
                filtered_stock.append(stock)
        
        return filtered_stock

In [6]:
# filter to verify the sector
def filter_sector(stock,sector):
    return stock.sector == sector

# filter to check price
def filter_price(stock, min_price, max_price):
    return min_price <= stock.price <= max_price

# filter metric based on the yahoo string and given operator
def filter_metric(stock, metric,operator,value):
    if metric not in stock.data:
        return False
    
    # converting the value to same unitsas metrics , if needed
    if 'B' in stock.data[metric]:
        stock.data[metric] = stock.data[metric].replace('B', '')
        value = float(value)/1e9
    elif 'M' in stock.data[metric]:
        stock.data[metric] = stock.data[metric].replace('M', '')
        value = float(value)/1e6
    elif '%' in stock.data[metric]:
        stock.data[metric] = stock.data[metric].replace('%', '')
        value = float(value)
    else:
        value =float(value)
        
    # check condition according to operator
    if operator == '>':
        return float(stock.data[metric]) > value
    elif operator == '>=':
        return float(stock.data[metric]) >=value
    elif operator == '<':
        return float(stock.data[metric]) < value
    elif operator == '<=':
        return float(stock.data[metric]) <=value
    elif operator == '==':
        return float(stock.data[metric]) ==value

In [7]:
# Get sp500 ticker and sector
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.find_all('tr')[1:]  # skip the header row

sp500 = []

for row in rows:
    cells = row.find_all('td')
    ticker = cells[0].text.strip()
    company = cells[1].text.strip()
    sector = cells[3].text.strip()
    sp500.append({'ticker': ticker, 'company': company, 'sector': sector})

def get_sp500_stocks():
    sp500_stocks = [Stock(stock["ticker"] , stock['sector']) for stock in sp500]
    return sp500_stocks

In [8]:
data = get_sp500_stocks()
data = get_sp500_stocks()
filters = [lambda stock: filter_sector(stock , "Application Software")]
screener = StockScreener(data , filters)
screener.add_data()

Gathered price of MMM!!!!
Gathered price of AOS!!!!
Gathered price of ABT!!!!
Gathered price of ABBV!!!!
Gathered price of ACN!!!!
Gathered price of ADBE!!!!
Gathered price of AMD!!!!
Gathered price of AES!!!!
Gathered price of AFL!!!!
Gathered price of A!!!!
Gathered price of APD!!!!
Gathered price of ABNB!!!!
Gathered price of AKAM!!!!
Gathered price of ALB!!!!
Gathered price of ARE!!!!
Gathered price of ALGN!!!!
Gathered price of ALLE!!!!
Gathered price of LNT!!!!
Gathered price of ALL!!!!
Gathered price of GOOGL!!!!
Gathered price of GOOG!!!!
Gathered price of MO!!!!
Gathered price of AMZN!!!!
Gathered price of AMCR!!!!
Gathered price of AEE!!!!
Gathered price of AAL!!!!
Gathered price of AEP!!!!
Gathered price of AXP!!!!
Gathered price of AIG!!!!
Gathered price of AMT!!!!
Gathered price of AWK!!!!
Gathered price of AMP!!!!
Gathered price of AME!!!!
Gathered price of AMGN!!!!
Gathered price of APH!!!!
Gathered price of ADI!!!!
Gathered price of ANSS!!!!
Gathered price of AON!!!!
Ga

ReadTimeout: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out. (read timeout=None)

In [ ]:

filtered_stocks = screener.apply_filters()
filtered_stocks

Price not available for MO
